In [1]:
import openai
from dotenv import load_dotenv
import os
import sys

sys.path.append("../find_similar_risk")
from embedding_providers import (
    OpenAIEmbeddingProvider,
    SentenceTransformerProvider,
    GeminiEmbeddingProvider,
)

env_path = "/Users/ford/Documents/coding/confidential/.env"
load_dotenv(env_path)
api_key = os.getenv("OPENAI_API_KEY")
assert api_key, "API key is missing"
# Set your OpenAI API key
openai.api_key = api_key

# Initialize embedding providers with caching
embedding_models = {
    "openai-large": OpenAIEmbeddingProvider(model_name="text-embedding-3-large"),
}

/Users/ford/Documents/coding_trae/cro_rmi_improvement_feature/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
test_input_sentence = "test_input_sentence"
result = embedding_models["openai-large"].get_embedding(test_input_sentence)

In [3]:
# test_tuple_key_dict
a_dict = {}
a_dict[tuple(sorted([1, 2]))] = "a"
a_dict[tuple(sorted([2, 1]))]

'a'

In [4]:
import json
import os

data_path = "company_risk_data.json"
data = json.load(open(data_path))
data[:3]

[{'company': 'Advance AQ',
  'risk_cat': 'Operational risk',
  'risk': 'After-sale service-related dissatisfaction',
  'risk_desc': 'The risk of customers experiencing dissatisfaction with the post-purchase support, assistance, or follow-up provided by the organization, leading to negative perceptions, reviews, or repurchase behavior. ขั้นตอนในการเคลมสินค้า เพื่อขอชดเชย มีหลายขั้นตอน ซึ่งขึ้นอยู่แต่ละปัญหาของสินค้า',
  'risk_level': 1.0,
  'rootcause': 'rootcause :ขั้นตอนการคืนสินค้า การแลกเปลี่ยน หรือการคืนเงินที่ซับซ้อน ยืดเยื้อ หรือไม่มีประสิทธิภาพ ส่งผลกระทบต่อความสามารถของลูกค้าในการหาทางแก้ไขปัญหาที่เกี่ยวข้องกับผลิตภัณฑ์\nrootcause_desc :ขั้นตอนการตรวจสอบการคืนสินค้าที่มีปัญหา หลายขั้นตอน และใช้เวลานาน',
  'process': 'process :Procurement,Research and Development\nprocess_desc :กระบวนการเคลมสินค้า กับ Supplier ต้องมีหลักฐานเอกสารประกอบ,ต้องทำการตรวจสอบจากหน่วยงาน R&D ก่อน'},
 {'company': 'Advance AQ',
  'risk_cat': 'Strategic Risk',
  'risk': 'Air pollution',
  'risk_desc': 'The

In [5]:
from itertools import combinations
from tqdm import tqdm

data_with_embedding = data.copy()
interest_columns = ["risk_desc", "rootcause", "process"]  # example list
all_combinations = []
for r in range(1, len(interest_columns) + 1):
    all_combinations.extend(combinations(interest_columns, r))
all_combinations = [list(c) for c in all_combinations]
all_combinations = [["risk"] + c for c in all_combinations]
all_combinations = [["risk"]] + all_combinations
for i in tqdm(range(len(data_with_embedding)), desc="Embedding rows"):
    for combinations in all_combinations:
        interest_columns = combinations
        key = tuple(sorted(interest_columns + ["embedding"]))
        data_list = []
        for column in interest_columns:
            data_list.append(data_with_embedding[i][column])
        try:
            data = "\n".join(data_list)
        except:
            print(data_list)
            raise
        data_with_embedding[i][key] = embedding_models["openai-large"].get_embedding(
            data
        )

Embedding rows: 100%|██████████| 305/305 [08:54<00:00,  1.75s/it]  


In [6]:
data_with_embedding[:1]

[{'company': 'Advance AQ',
  'risk_cat': 'Operational risk',
  'risk': 'After-sale service-related dissatisfaction',
  'risk_desc': 'The risk of customers experiencing dissatisfaction with the post-purchase support, assistance, or follow-up provided by the organization, leading to negative perceptions, reviews, or repurchase behavior. ขั้นตอนในการเคลมสินค้า เพื่อขอชดเชย มีหลายขั้นตอน ซึ่งขึ้นอยู่แต่ละปัญหาของสินค้า',
  'risk_level': 1.0,
  'rootcause': 'rootcause :ขั้นตอนการคืนสินค้า การแลกเปลี่ยน หรือการคืนเงินที่ซับซ้อน ยืดเยื้อ หรือไม่มีประสิทธิภาพ ส่งผลกระทบต่อความสามารถของลูกค้าในการหาทางแก้ไขปัญหาที่เกี่ยวข้องกับผลิตภัณฑ์\nrootcause_desc :ขั้นตอนการตรวจสอบการคืนสินค้าที่มีปัญหา หลายขั้นตอน และใช้เวลานาน',
  'process': 'process :Procurement,Research and Development\nprocess_desc :กระบวนการเคลมสินค้า กับ Supplier ต้องมีหลักฐานเอกสารประกอบ,ต้องทำการตรวจสอบจากหน่วยงาน R&D ก่อน',
  ('embedding',
   'risk'): array([ 0.00843114, -0.01816072, -0.01205196, ..., -0.00681027,
          0.00

In [7]:
# save data_with_embedding to pickle
import pickle

with open("company_risk_data_with_embedding.pkl", "wb") as f:
    pickle.dump(data_with_embedding, f)